In [1]:
from torchvision import transforms
import cv2

input_size = 256

idx = 0000
root_dataset = "../destijl_dataset/"
design_path = root_dataset + '00_preview/'+str(idx)+".png",
image_xml = root_dataset + "xmls/" + '02_image/'+str(idx)+".xml",
text_xml = root_dataset + "xmls/" + '04_text/'+str(idx)+".xml",

transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
    ])